In [3]:
import os
import pandas as pd
import numpy as np
from tqdm  import tqdm
import pickle
import re
import glob

# Data preparation

In [11]:
# Set the directory path where the pickle files are located
directory_path = os.getcwd() + '/ABC'

################# PATH FOR JONAS ################# directory_path = os.getcwd() + '\\ABC'

# Get a list of all the pickle files in the directory
pickle_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.pkl') and f.startswith('full') == False ]

# Load and concatenate the data from each file
# create the empty dataframe with 13 columns

concatenated_data = []
for file in tqdm(pickle_files):
    with open(file, 'rb') as f:
        staged_data = pickle.load(f)
        concatenated_data.append(staged_data)

data = pd.concat(concatenated_data, axis=0)
# adding id
data['id'] = data.index
cols = data.columns.tolist()
cols = cols[-1:] + cols[:-1]
data = data[cols]

100%|██████████| 9/9 [00:00<00:00, 301.70it/s]


In [12]:
with open(directory_path + '/full_data.pkl', 'rb') as f:
    data = pickle.load(f)
data.head(3)
data.shape

(13730, 17)

In [13]:
# arts is incorrectly labelled as comedy, also good naming, and missing values

data['category'].replace('Comedy', 'Arts', inplace = True)
data.rename({'more':'directors_actors'}, axis = 1, inplace = True)
data.replace(['No data found', 'No more information found', 
              'No title found', 'No tags found', 'No image found', 'No description found'], np.nan, inplace = True)

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13730 entries, 0 to 2528
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                13730 non-null  int64  
 1   category          13730 non-null  object 
 2   title             13730 non-null  object 
 3   series            13279 non-null  object 
 4   episode_name      13730 non-null  object 
 5   description       13730 non-null  object 
 6   description2      13730 non-null  object 
 7   tags              13730 non-null  object 
 8   image             13730 non-null  object 
 9   directors_actors  4384 non-null   object 
 10  tags2             13730 non-null  object 
 11  publication_date  13730 non-null  object 
 12  rating            11803 non-null  object 
 13  duration_sec      13730 non-null  float64
 14  season            13730 non-null  object 
 15  episode           13730 non-null  object 
 16  episode_title     13730 non-null  object 

In [14]:
cols = list(data.columns)
# dropping nans where it is necessary
unwanted = ['directors_actors', 'rating', 'series']
cols = [x for x in cols if x not in unwanted]

clean = data.dropna(subset = cols)
clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13730 entries, 0 to 2528
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                13730 non-null  int64  
 1   category          13730 non-null  object 
 2   title             13730 non-null  object 
 3   series            13279 non-null  object 
 4   episode_name      13730 non-null  object 
 5   description       13730 non-null  object 
 6   description2      13730 non-null  object 
 7   tags              13730 non-null  object 
 8   image             13730 non-null  object 
 9   directors_actors  4384 non-null   object 
 10  tags2             13730 non-null  object 
 11  publication_date  13730 non-null  object 
 12  rating            11803 non-null  object 
 13  duration_sec      13730 non-null  float64
 14  season            13730 non-null  object 
 15  episode           13730 non-null  object 
 16  episode_title     13730 non-null  object 

In [16]:
films = clean[(clean['series'].isna()) & (clean['category'] != 'News')]
films.shape

(419, 17)

# Creating special columns for episodes

In [33]:
seasons = []
eps = []
ep_descs = []

for index, row in clean.iterrows():
    i = index
    txt = row.episode_name
    try:
        season = re.findall('Series [0-9]{1,2}', txt)[0]
        season = re.findall('[0-9]+', season)[0]
        seasons.append(season)
    except:
        seasons.append(np.nan)
    try: 
        ep = re.findall('Episode [0-9]{1,2}', txt)[0]
        ep = re.findall('[0-9]+', ep)[0]
        eps.append(ep)
    except:
        eps.append('')
    desc = re.sub('[0-9]+', '', txt)
    desc = desc.replace('Series', '')
    desc = desc.replace('Episode', '')
    ep_descs.append(desc)

In [34]:
clean = clean.assign(season=seasons, episode=eps, episode_title=ep_descs)
clean.head(3)

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,0,Panel Discussion,Spicks And Specks,Spicks And Specks,"Series 9 Missy Higgins, Dave O'Neil, Yeo & Jud...","Join Adam Hills, Myf Warhurst and Alan Brough ...",Missy Higgins and Dave O'Neil help Alan battle...,"[ABC TV, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/le/LE200...,"Hosts Adam Hills, Myf Warhurst, Alan Brough","[abc1, australia, panel, music, culture, enter...",2021-06-20 20:27:00,PG,2701.0,9,,"Missy Higgins, Dave O'Neil, Yeo & Judith Lucy"
1,1,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 4,Rob Brydon is back in the host's chair for ano...,Host Rob Brydon and team captains Lee Mack and...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-02-08 20:30:35,PG,1737.0,13,4,
2,2,Panel Discussion,Whovians,Whovians,Series 3 Episode 9,"Join Rove McManus, Tegan Higginbotham and Bajo...","Rove McManus, Tegan Higginbotham and Bajo are ...","[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/le/LE191...,"Hosts Rove McManus, Tegan Higginbotham, Steven...","[abc2, aussie, comedy, panel, entertainment, q...",2020-03-05 21:38:00,PG,2090.0,3,9,


In [19]:
films = clean[(clean['series'].isna()) & (clean['category'] != 'News') & (clean['category'] != 'Panel Discussion')]

with open(directory_path + '/full_movies.pkl', 'wb') as f:
    pickle.dump(films, f)

In [20]:
# Write the concatenated data to a new file
with open(directory_path + '/full_data.pkl', 'wb') as f:
    pickle.dump(clean, f)

# Creating a new dataframe with only the series

In [9]:
series_raw = clean[(clean['series'].isna()==False)]
series_raw.head(3)

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,0,Panel Discussion,Spicks And Specks,Spicks And Specks,"Series 9 Missy Higgins, Dave O'Neil, Yeo & Jud...","Join Adam Hills, Myf Warhurst and Alan Brough ...",Missy Higgins and Dave O'Neil help Alan battle...,"[ABC TV, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/le/LE200...,"Hosts Adam Hills, Myf Warhurst, Alan Brough","[abc1, australia, panel, music, culture, enter...",2021-06-20 20:27:00,PG,2701.0,9,,"Missy Higgins, Dave O'Neil, Yeo & Judith Lucy"
1,1,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 4,Rob Brydon is back in the host's chair for ano...,Host Rob Brydon and team captains Lee Mack and...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-02-08 20:30:35,PG,1737.0,13,4,
2,2,Panel Discussion,Whovians,Whovians,Series 3 Episode 9,"Join Rove McManus, Tegan Higginbotham and Bajo...","Rove McManus, Tegan Higginbotham and Bajo are ...","[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/le/LE191...,"Hosts Rove McManus, Tegan Higginbotham, Steven...","[abc2, aussie, comedy, panel, entertainment, q...",2020-03-05 21:38:00,PG,2090.0,3,9,


In [47]:
series = list(test.series)
titles = list(test.title)

non_matches = []

for i in range(len(series)):
    if series[i] != titles[i]:
        non_matches.append(i)

len(non_matches)

179

In [49]:
series = list(data.series)
titles = list(data.title)

non_matches = []

for i in range(len(series)):
    if series[i] != titles[i]:
        non_matches.append(i)
        
len(non_matches)

1226

In [52]:
data.iloc[non_matches].category.value_counts()

News                229
Arts                204
Documentary         168
Education           146
Movies              143
Drama               138
Family              133
Panel Discussion     34
Kids                 31
Name: category, dtype: int64

In [87]:
test.iloc[3021]

category                                                       Family
title                                                   The Next Step
series                                   The Next Step: Bonus Content
episode_name             Series 7 200th Episode Celebration: Season 6
description         An unexpected opportunity divides The Next Ste...
description2        Briar Nolet (Richelle) and Myles Erlick (Noah)...
tags                                                 [ABC ME, FAMILY]
image               https://cdn.iview.abc.net.au/thumbs/i/zw/ZW313...
directors_actors    Cast Alexandra Beaton, Brittany Raymond, Isaac...
tags2                                      [abc3, 3dance, abc3-faves]
publication_date                                  2022-02-01 06:00:00
rating                                                              G
duration_sec                                                    310.0
Name: 7151, dtype: object

In [ ]:
## all the shows that don't have the same title and series name are some kind of bonus content for the same show: do we want to deal with this?

In [72]:
test.title

0                     Bangarra's World
2        The Story of Film: An Odyssey
4         Julia Zemiro's Home Delivery
5                        Going Country
6        The Story of Film: An Odyssey
                     ...              
13872                               QI
13874              Question Everything
13877              Would I Lie To You?
13879                    Mock The Week
13883              Would I Lie To You?
Name: title, Length: 3678, dtype: object

In [104]:
filtered = frame.dropna(subset = 'description2')

desc = list(filtered.description2)

In [106]:
x = []

for i in range(len(desc)):
    if "found" in desc[i]:
        x.append(i)
        
x

[29,
 43,
 48,
 57,
 61,
 118,
 129,
 138,
 151,
 224,
 227,
 245,
 309,
 338,
 359,
 411,
 521,
 531,
 591,
 593,
 619,
 620,
 657,
 675,
 695,
 706,
 710,
 736,
 781,
 782,
 824,
 872,
 873,
 930,
 957,
 961,
 974,
 997,
 1008,
 1076,
 1108,
 1134,
 1197,
 1200,
 1218,
 1220,
 1269,
 1294,
 1337,
 1370,
 1398,
 1500,
 1539,
 1547,
 1591,
 1592,
 1669,
 1756,
 1767,
 1791,
 1835,
 1898,
 1935,
 1943,
 1964,
 1993,
 2042,
 2053,
 2055,
 2056,
 2081,
 2150,
 2152,
 2163,
 2167,
 2168,
 2267,
 2295,
 2303,
 2322,
 2332,
 2338,
 2349,
 2384,
 2408,
 2416,
 2417,
 2420,
 2472,
 2510,
 2529,
 2530,
 2531,
 2564,
 2596,
 2616,
 2624,
 2628,
 2629,
 2664,
 2684,
 2762,
 2770,
 2798,
 2841,
 2844,
 2861,
 2864,
 2994,
 3011,
 3028,
 3031,
 3082,
 3102,
 3111,
 3124,
 3191,
 3196,
 3236,
 3420,
 3549,
 3760,
 4075,
 4115,
 4131,
 4223,
 4291,
 4317,
 4459,
 4807,
 4864,
 4926,
 5096,
 5233,
 5309,
 5330,
 5450,
 5528,
 5633,
 5751,
 5977,
 6460,
 6558,
 6588,
 6937,
 7068,
 7551,
 7999,
 8375,


In [109]:
filtered.iloc[521].description2

"Saba is the founder and Manager of Mu'ooz Restaurant. The not-for-profit social enterprise has provided employment and training opportunities for women who've arrived as refugees."

In [110]:
[filtered.iloc[i].description2 for i in x]

["Find out why Melbourne suddenly found the need to build a gallery of great significance on St Kilda Road, and how the site went from being a pile of rubble and ideas to become one of the world's most visited art museums.",
 'The owner of two sketchbooks found in a shed in France ask Fiona Bruce and Philip Mould to help prove they are the work of a young Toulouse Lautrec.',
 'As founders of Lonely Planet, Tony Wheeler and his wife Maureen cornered a market and built a multi-million-dollar travel guide empire. The business has now been sold, but Tony remains a tireless globetrotter.',
 'Rosalie Gascoigne captured the Australian landscape by assembling objects she found within it, finding beauty in weathered discarded road signs and farming equipment. Her distinct representations of Australia and her contribution to art gained her an Order of Australia in 1994.',
 'Tahnee founded Hand-n-Hand Peer Support in 2020 to assist healthcare workers experiencing wellbeing or mental health diffic

In [139]:
frame

,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec
0,Arts,Bangarra's World,Bangarra's World,Return to Country (Yirrkala),Join the dancers and creatives during Bangarra...,"In Yirrkala, Northeast Arnhem Land, Bangarra s...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,G,626.0
1,Arts,Mission Songs Project,Mission Songs Project,Surrare,Jessie Lloyd's Mission Songs Project revives s...,"A song from the Torres Strait Islands, 'Surrar...","[ABC TV, DOCUMENTARY, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/ip/IP206...,NaN,"[abc1, docs, factual, arts, indigenous, music,...",2020-11-07 07:00:00,G,313.0
2,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 5 Post-War Cinema,A worldwide guided tour of the greatest movies...,1939-1952 The trauma of war led to more daring...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,M,3728.0
3,Arts,Ballet Now,NaN,Ballet Now,This film captures the creative process behind...,This film captures the creative process behind...,"[ABC TV Plus, DOCUMENTARY, ARTS & CULTURE, FAM...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW269...,NaN,"[abc2, docs, factual, arts, ballet, dance, cul...",2021-06-30 21:59:27,M,3555.0
4,Arts,Julia Zemiro's Home Delivery,Julia Zemiro's Home Delivery,Series 3 Waleed Aly,Julia Zemiro invites notable personalities to ...,"Waleed Aly is a lawyer, writer, columnist and ...","[ABC TV, ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/le/LE146...,Host Julia Zemiro,"[abc1, abc2, docs, people, entertainment, inte...",2021-06-04 07:00:00,G,1619.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13880,Panel Discussion,Q+A,Q+A,Budget Special,Q+A is back to get more of the answers to your...,"David Speers is joined by Barnaby Joyce, Jim C...","[ABC TV, ABC NEWS, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,"Host Virginia Trioli, David Speers, Stan Grant","[abc1, news, panel, current-affairs, politics,...",2022-03-31 21:39:55,NaN,3665.0
13881,Panel Discussion,One Plus One,One Plus One,Series 2019 Murray Cook,Some of your favourite ABC presenters bring yo...,"Seven years ago, Murray Cook hung up his red s...","[ABC TV, ARTS & CULTURE, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC191...,NaN,"[abc1, news24, arts, panel, talks, people, int...",2019-03-15 13:30:00,NaN,1680.0
13882,Panel Discussion,Q+A,Q+A,"The Vaccine, the Church, and the State of Poli...",Q+A is back to get more of the answers to your...,"The Government's ""women's ministry"" meets for ...","[ABC TV, ABC NEWS, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,"Host Virginia Trioli, David Speers, Stan Grant","[abc1, news, panel, current-affairs, politics,...",2021-04-08 21:31:57,NaN,3443.0
13883,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 11 More Unseen Bits,Rob Brydon is back in the host's chair for ano...,More unseen clips from this series of Would I ...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-03-29 20:29:24,PG,1725.0


In [141]:
frame.episode_name

0                             Return to Country (Yirrkala)
1                                                  Surrare
2                                Episode 5 Post-War Cinema
3                                               Ballet Now
4                                      Series 3 Waleed Aly
                               ...                        
13880                                       Budget Special
13881                              Series 2019 Murray Cook
13882    The Vaccine, the Church, and the State of Poli...
13883                Series 13 Episode 11 More Unseen Bits
13884        Boats, Boys Clubs and Businesses on the Brink
Name: episode_name, Length: 13885, dtype: object

In [143]:
frame.iterrows()[0]

TypeError: 'generator' object is not subscriptable

In [144]:
row = next(frame.iterrows())[1]

In [147]:
row

category                                                            Arts
title                                                   Bangarra's World
series                                                  Bangarra's World
episode_name                                Return to Country (Yirrkala)
description            Join the dancers and creatives during Bangarra...
description2           In Yirrkala, Northeast Arnhem Land, Bangarra s...
tags                   [ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...
image                  https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...
directors_actors                       Director Nel Minchin, Wayne Blair
tags2                  [abc1, abc2, indigenous, abcarts, arts, docs, ...
publication_date                                     2021-07-07 22:00:00
rating                                                                 G
duration_sec                                                       626.0
season                                             

In [154]:
for i in range(len(frame)):
    row = next(frame.iterrows())[i + 1]
    print(row.category)

Arts


IndexError: tuple index out of range

In [149]:
frame.iterrows()

<generator object DataFrame.iterrows at 0x00000293C241C510>

In [150]:
for row in frame.iterrows():
    print(row.title)

AttributeError: 'tuple' object has no attribute 'title'

In [71]:
clean.drop(columns = ['season', 'episode', 'episode_title'], inplace = True)

/var/folders/mv/3y_h14td39s6m0zfltc84cb00000gq/T/ipykernel_34086/1405913815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean.drop(columns = ['season', 'episode', 'episode_title'], inplace = True)


In [68]:
clean.columns

Index(['id', 'category', 'title', 'series', 'episode_name', 'description',
       'description2', 'tags', 'image', 'directors_actors', 'tags2',
       'publication_date', 'rating', 'duration_sec', 'season', 'episode',
       'episode_title'],
      dtype='object')

In [224]:
clean.category.unique()

array(['Arts', 'Documentary', 'Drama', 'Education', 'Family', 'Kids',
       'Movies', 'News', 'Panel Discussion'], dtype=object)

In [255]:
display(clean.loc[(clean['category'] == 'Arts') & (clean['duration_sec'] > 3600)])

,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
2,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 5 Post-War Cinema,A worldwide guided tour of the greatest movies...,1939-1952 The trauma of war led to more daring...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,M,3728.0,,5,Post-War Cinema
6,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 15 Cinema Today and the Future,A worldwide guided tour of the greatest movies...,"The 2000s Cinema got more serious after 9/11, ...","[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,MA,3754.0,,15,Cinema Today and the Future
12,Arts,Freddie Mercury: The Final Act,NaN,Freddie Mercury: The Final Act,The extraordinary story of Freddie Mercury's b...,The extraordinary story of Freddie Mercury's b...,"[ABC TV, DOCUMENTARY, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW324...,NaN,"[abc1, docs, arts, factual, culture, music, ro...",2022-03-15 20:31:03,M,5377.0,,,Freddie Mercury: The Final Act
32,Arts,Pina,NaN,Pina,"From the acclaimed filmmaker Wim Wenders, Pina...","From the acclaimed filmmaker Wim Wenders, Pina...","[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zx/ZX958...,Director Wim Wenders,"[abc1, abc2, abcarts, arts, docs, factual, per...",2021-06-11 07:00:00,G,5949.0,,,Pina
39,Arts,Becoming Jane,NaN,Becoming Jane,"When feisty, 20-year-old, budding novelist, Ja...","When feisty, 20-year-old, budding novelist, Ja...","[ABC TV, ABC TV Plus, DRAMA, ARTS & CULTURE, M...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,Director Julian Jarrold,"[abc1, abc2, drama, period-drama, 1700s, arts,...",2021-06-01 07:00:00,PG,6915.0,,,Becoming Jane
110,Arts,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,NaN,"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,,,Romeo & Juliet
126,Arts,The Merry Widow,NaN,The Merry Widow,"The Merry Widow is a lively tale of love, mone...","The Merry Widow is a lively tale of love, mone...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,NaN,"[abc2, arts, abcarts, ballet, opera, entertain...",2021-06-18 07:00:00,G,5877.0,,,The Merry Widow
143,Arts,Concert For Ukraine,NaN,Concert For Ukraine,"Fast-tracked from the UK, join some of the big...","Fast-tracked from the UK, join some of the big...","[ABC TV Plus, ABC TV, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW332...,"Hosts Roman Kemp, Marvin Humes, Emma Bunton","[abc2, abc1, arts, abcarts, music, concert, uk...",2022-03-30 20:31:45,PG,6065.0,,,Concert For Ukraine
170,Arts,INXS: Live Baby Live,NaN,INXS: Live Baby Live,"On 13th July 1991, INXS were filmed in concert...","On 13th July 1991, INXS were filmed in concert...","[ABC TV Plus, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW316...,NaN,"[abc2, uk, arts, entertainment, concert, music...",2021-11-26 20:32:24,M,5892.0,,,INXS: Live Baby Live
180,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 14 New American Independents & the Dig...,A worldwide guided tour of the greatest movies...,The 1990s Digital filmmaking allowed for the c...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,MA,3724.0,,14,New American Independents & the 

In [257]:
clean[clean['category'] == 'Movies']

,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
12092,Movies,The Way We Were,NaN,The Way We Were,A love story that begins with the attraction o...,A love story that begins with the attraction o...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zy/ZY933...,Director Sydney Pollack,"[abc1, abc2, drama, classic, film, feature-len...",2021-06-25 07:00:00,PG,6802.0,,,The Way We Were
12093,Movies,Starman,NaN,Starman,An alien stranded on Earth clones himself into...,An alien stranded on Earth clones himself into...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW301...,Director John Carpenter,"[abc1, abc2, drama, sci-fi, fantasy, romance, ...",2021-06-25 07:00:00,PG,6612.0,,,Starman
12094,Movies,Mr Deeds Goes To Town,NaN,Mr Deeds Goes To Town,A small-town poet inherits a vast fortune and ...,A small-town poet inherits a vast fortune and ...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW302...,"Cast Gary Cooper, Jean Arthur, Douglas Dumbril...","[abc1, abc2, drama, comedy-drama, film, featur...",2021-06-25 07:00:00,G,6649.0,,,Mr Deeds Goes To Town
12095,Movies,On The Road,NaN,On The Road,Aspiring writer Sal Paradise has his world roc...,Aspiring writer Sal Paradise has his world roc...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW291...,Director Walter Salles,"[abc1, abc2, usa, drama, adaptation, adventure...",2021-06-01 07:00:00,MA,8014.0,,,On The Road
12096,Movies,The Boys Are Back,NaN,The Boys Are Back,"Journalist Joe Warr and his two sons, from dif...","Journalist Joe Warr and his two sons, from dif...","[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,Director Scott Hicks,"[abc1, abc2, drama, comedy-drama, film, single...",2021-06-11 07:00:00,M,5950.0,,,The Boys Are Back
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12230,Movies,Griff The Invisible,NaN,Griff The Invisible,"Griff, office worker by day, superhero by nigh...","Griff, office worker by day, superhero by nigh...","[ABC TV, ABC TV Plus, MOVIES, COMEDY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW277...,Director Leon Ford,"[abc1, abc2, film, feature-length, sci-fi, com...",2021-06-01 07:00:00,M,5307.0,,,Griff The Invisible
12231,Movies,Murray Whelan: Stiff,NaN,Murray Whelan: Stiff,"Murray Whelan, a young political advisers, mus...","Murray Whelan, a young political advisers, mus...","[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,Director John Clarke,"[abc1, australia, drama, crime, crime-drama, 2...",2021-06-25 07:00:00,M,5567.0,,,Murray Whelan: Stiff
12232,Movies,13 Assassins,NaN,13 Assassins,When the sadistic excesses of Lord Matsudaira ...,When the sadistic excesses of Lord Matsudaira ...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,Director Takashi Miike,"[abc1, abc2, action, drama, film, period-drama...",2021-06-01 07:00:00,MA,7157.0,,,Assassins
12233,Movies,Train To Busan,NaN,Train To Busan,"As a zombie outbreak sweeps South Korea, a fat...","As a zombie outbreak sweeps South Korea, a fat...","[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Director Sang-ho Yeon,"[abc1, abc2, drama, action, horror, thriller, ...",2021-06-01 07:00:00,MA,6760.0,,,Train To Busan


In [248]:
clean.loc[np.where((clean['category'] == 'Arts') & (clean['duration_sec'] > 3600))]

,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
2,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 5 Post-War Cinema,A worldwide guided tour of the greatest movies...,1939-1952 The trauma of war led to more daring...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,M,3728.0,,5,Post-War Cinema
6,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 15 Cinema Today and the Future,A worldwide guided tour of the greatest movies...,"The 2000s Cinema got more serious after 9/11, ...","[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,MA,3754.0,,15,Cinema Today and the Future
12,Arts,Freddie Mercury: The Final Act,NaN,Freddie Mercury: The Final Act,The extraordinary story of Freddie Mercury's b...,The extraordinary story of Freddie Mercury's b...,"[ABC TV, DOCUMENTARY, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW324...,NaN,"[abc1, docs, arts, factual, culture, music, ro...",2022-03-15 20:31:03,M,5377.0,,,Freddie Mercury: The Final Act
32,Arts,Pina,NaN,Pina,"From the acclaimed filmmaker Wim Wenders, Pina...","From the acclaimed filmmaker Wim Wenders, Pina...","[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zx/ZX958...,Director Wim Wenders,"[abc1, abc2, abcarts, arts, docs, factual, per...",2021-06-11 07:00:00,G,5949.0,,,Pina
39,Arts,Becoming Jane,NaN,Becoming Jane,"When feisty, 20-year-old, budding novelist, Ja...","When feisty, 20-year-old, budding novelist, Ja...","[ABC TV, ABC TV Plus, DRAMA, ARTS & CULTURE, M...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,Director Julian Jarrold,"[abc1, abc2, drama, period-drama, 1700s, arts,...",2021-06-01 07:00:00,PG,6915.0,,,Becoming Jane
109,Arts,Compass,Compass,Series 34 Material Gain,Compass explores the interface between religio...,"The Red Cross, St Vincent de Paul and The Salv...","[ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rn/RN191...,Host Geraldine Doogue,"[abc2, australia, factual, docs, religion, eth...",2020-05-03 19:01:34,G,1531.0,34,,Material Gain
125,Arts,Songs Of Praise: Gospel Singer Of The Year 202...,NaN,Songs Of Praise: Gospel Singer Of The Year 202...,This celebration of Gospel music is hosted by ...,This celebration of Gospel music is hosted by ...,"[ABC TV, ARTS & CULTURE, DOCUMENTARY, LIFESTYLE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW323...,NaN,"[abc1, religion, arts, music, culture, docs, l...",2022-03-13 11:29:19,G,1781.0,,,Songs Of Praise: Gospel Singer Of The Year Final
142,Arts,Barrie Cassidy's One Plus One,Barrie Cassidy's One Plus One,Series 2021 Fran Kelly,Barrie Cassidy returns for a new season of One...,"After 17 years of early morning alarms, 2021 w...","[ABC TV, ARTS & CULTURE, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC213...,Host Barrie Cassidy,"[abc1, news24, arts, panel, talks, people, int...",2021-12-02 20:31:30,NaN,1770.0,20,,Fran Kelly
169,Arts,Rewind,Rewind,Series 5 Tim Brooke-Taylor: The Plight of Gnomes,This archive-based series is a kaleidoscope of...,"Tim Brooke-Taylor, one-third of the comedy sen...","[ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/ac/AC192...,NaN,"[abcarts, arts, rewind, archive, arts-featured]",2020-04-16 18:00:00,NaN,170.0,5,,Tim Brooke-Taylor: The Plight of Gnomes
179,Arts,Anh's Brush With Fame,Anh's Brush With Fame,Series 6 Manu Feildel,Anh Do paints some of Australia's most famous ...,French chef Manu Feildel's passion for enterta...,"[ABC TV, ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO203...,Host Anh Do,"[abc1, abc2, docs, people, enter

In [233]:
clean[clean['category'] == 'Arts'] & clean['duration_sec'] > 3600

SyntaxError: unmatched ']' (<ipython-input-233-756d267c942c>, line 1)

In [230]:
 clean[clean[clean['category'] == Arts]] & clean['duration_sec'] > 3600]

SyntaxError: invalid syntax (<ipython-input-230-0c00fb8c90b0>, line 1)

In [189]:
clean

,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_description
0,Arts,Bangarra's World,Bangarra's World,Return to Country (Yirrkala),Join the dancers and creatives during Bangarra...,"In Yirrkala, Northeast Arnhem Land, Bangarra s...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,G,626.0,,,
1,Arts,Mission Songs Project,Mission Songs Project,Surrare,Jessie Lloyd's Mission Songs Project revives s...,"A song from the Torres Strait Islands, 'Surrar...","[ABC TV, DOCUMENTARY, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/ip/IP206...,NaN,"[abc1, docs, factual, arts, indigenous, music,...",2020-11-07 07:00:00,G,313.0,,,
2,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 5 Post-War Cinema,A worldwide guided tour of the greatest movies...,1939-1952 The trauma of war led to more daring...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,M,3728.0,,,
3,Arts,Ballet Now,NaN,Ballet Now,This film captures the creative process behind...,This film captures the creative process behind...,"[ABC TV Plus, DOCUMENTARY, ARTS & CULTURE, FAM...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW269...,NaN,"[abc2, docs, factual, arts, ballet, dance, cul...",2021-06-30 21:59:27,M,3555.0,,,
4,Arts,Julia Zemiro's Home Delivery,Julia Zemiro's Home Delivery,Series 3 Waleed Aly,Julia Zemiro invites notable personalities to ...,"Waleed Aly is a lawyer, writer, columnist and ...","[ABC TV, ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/le/LE146...,Host Julia Zemiro,"[abc1, abc2, docs, people, entertainment, inte...",2021-06-04 07:00:00,G,1619.0,Series 3,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13880,Panel Discussion,Q+A,Q+A,Budget Special,Q+A is back to get more of the answers to your...,"David Speers is joined by Barnaby Joyce, Jim C...","[ABC TV, ABC NEWS, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,"Host Virginia Trioli, David Speers, Stan Grant","[abc1, news, panel, current-affairs, politics,...",2022-03-31 21:39:55,NaN,3665.0,,,
13881,Panel Discussion,One Plus One,One Plus One,Series 2019 Murray Cook,Some of your favourite ABC presenters bring yo...,"Seven years ago, Murray Cook hung up his red s...","[ABC TV, ARTS & CULTURE, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC191...,NaN,"[abc1, news24, arts, panel, talks, people, int...",2019-03-15 13:30:00,NaN,1680.0,Series 2019,,
13882,Panel Discussion,Q+A,Q+A,"The Vaccine, the Church, and the State of Poli...",Q+A is back to get more of the answers to your...,"The Government's ""women's ministry"" meets for ...","[ABC TV, ABC NEWS, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,"Host Virginia Trioli, David Speers, Stan Grant","[abc1, news, panel, current-affairs, politics,...",2021-04-08 21:31:57,NaN,3443.0,,,
13883,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 11 More Unseen Bits,Rob Brydon is back in the host's chair for ano...,More unseen clips from this series of Would I ...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-03-29 20:29:24,PG,1725.0,Series 13,,


In [187]:
clean.assign(season = seasons, inplace = True)

clean.head(10)

,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_description
0,Arts,Bangarra's World,Bangarra's World,Return to Country (Yirrkala),Join the dancers and creatives during Bangarra...,"In Yirrkala, Northeast Arnhem Land, Bangarra s...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,G,626.0,,,
1,Arts,Mission Songs Project,Mission Songs Project,Surrare,Jessie Lloyd's Mission Songs Project revives s...,"A song from the Torres Strait Islands, 'Surrar...","[ABC TV, DOCUMENTARY, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/ip/IP206...,NaN,"[abc1, docs, factual, arts, indigenous, music,...",2020-11-07 07:00:00,G,313.0,,,
2,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 5 Post-War Cinema,A worldwide guided tour of the greatest movies...,1939-1952 The trauma of war led to more daring...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,M,3728.0,,,
3,Arts,Ballet Now,NaN,Ballet Now,This film captures the creative process behind...,This film captures the creative process behind...,"[ABC TV Plus, DOCUMENTARY, ARTS & CULTURE, FAM...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW269...,NaN,"[abc2, docs, factual, arts, ballet, dance, cul...",2021-06-30 21:59:27,M,3555.0,,,
4,Arts,Julia Zemiro's Home Delivery,Julia Zemiro's Home Delivery,Series 3 Waleed Aly,Julia Zemiro invites notable personalities to ...,"Waleed Aly is a lawyer, writer, columnist and ...","[ABC TV, ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/le/LE146...,Host Julia Zemiro,"[abc1, abc2, docs, people, entertainment, inte...",2021-06-04 07:00:00,G,1619.0,,,
5,Arts,Going Country,Going Country,Series 1 Episode 1,Justine Clarke takes a road trip into the hear...,Justine Clarke connects with some of Australia...,"[ABC TV, DOCUMENTARY, ARTS & CULTURE, LIFESTYL...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC201...,Host Justine Clarke,"[abc1, docs, arts, factual, lifestyle, country...",2021-11-02 20:30:14,PG,3244.0,,,
6,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 15 Cinema Today and the Future,A worldwide guided tour of the greatest movies...,"The 2000s Cinema got more serious after 9/11, ...","[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,MA,3754.0,,,
7,Arts,Julia Zemiro's Home Delivery,Julia Zemiro's Home Delivery,Series 7 Costa Georgiadis,Julia Zemiro invites notable personalities to ...,Julia Zemiro travels with Gardening Australia ...,"[ABC TV, ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/le/LE186...,Host Julia Zemiro,"[abc1, abc2, docs, people, entertainment, inte...",2019-10-09 20:31:55,PG,1677.0,,,
8,Arts,Compass,Compass,Series 33 Women Who Do,Compass explores the interface between religio...,"Before women had the vote in Australia, Anglic...","[ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rn/RN181...,Host Geraldine Doogue,"[abc2, australia, factual, docs, religion, eth...",2020-01-03 17:00:00,G,1634.0,,,
9,Arts,Stan Grant's One Plus One,Stan Grant's One Plus One,Kodie Bedford,Stan Grant presents a series of in-depth conve...,Kodie Bedford is a storyteller who aims to tel...,[ABC TV],https://cdn.iview.abc.net.au/thumbs/i/nc/NC203...,Host Stan Grant,"[abc1, news24, factual, one-plus-one, intervie...",2021-01-18 20:32:05,NaN,1700.0,,,


In [179]:
for i in range(len(extracted_episodes_dict)):
    row = extracted_episodes_dict[i]
    clean.loc[i].season = row[0]
    clean.loc[i].episode = row[1]
    clean.loc[i].episode_description = row[2]
    
clean

<ipython-input-179-a83024858d83>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean.loc[i].season = row[0]
<ipython-input-179-a83024858d83>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean.loc[i].episode = row[1]
<ipython-input-179-a83024858d83>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean.loc[i].episode_description = row[2]


KeyError: 85

In [169]:
frame

,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_description
0,Arts,Bangarra's World,Bangarra's World,Return to Country (Yirrkala),Join the dancers and creatives during Bangarra...,"In Yirrkala, Northeast Arnhem Land, Bangarra s...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,G,626.0,,,
1,Arts,Mission Songs Project,Mission Songs Project,Surrare,Jessie Lloyd's Mission Songs Project revives s...,"A song from the Torres Strait Islands, 'Surrar...","[ABC TV, DOCUMENTARY, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/ip/IP206...,NaN,"[abc1, docs, factual, arts, indigenous, music,...",2020-11-07 07:00:00,G,313.0,,,
2,Arts,The Story of Film: An Odyssey,The Story of Film: An Odyssey,Episode 5 Post-War Cinema,A worldwide guided tour of the greatest movies...,1939-1952 The trauma of war led to more daring...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,Host Mark Cousins,"[abc1, abc2, arts, abcarts, docs, movies, retr...",2021-06-11 07:00:00,M,3728.0,,,
3,Arts,Ballet Now,NaN,Ballet Now,This film captures the creative process behind...,This film captures the creative process behind...,"[ABC TV Plus, DOCUMENTARY, ARTS & CULTURE, FAM...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW269...,NaN,"[abc2, docs, factual, arts, ballet, dance, cul...",2021-06-30 21:59:27,M,3555.0,,,
4,Arts,Julia Zemiro's Home Delivery,Julia Zemiro's Home Delivery,Series 3 Waleed Aly,Julia Zemiro invites notable personalities to ...,"Waleed Aly is a lawyer, writer, columnist and ...","[ABC TV, ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/le/LE146...,Host Julia Zemiro,"[abc1, abc2, docs, people, entertainment, inte...",2021-06-04 07:00:00,G,1619.0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13880,Panel Discussion,Q+A,Q+A,Budget Special,Q+A is back to get more of the answers to your...,"David Speers is joined by Barnaby Joyce, Jim C...","[ABC TV, ABC NEWS, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,"Host Virginia Trioli, David Speers, Stan Grant","[abc1, news, panel, current-affairs, politics,...",2022-03-31 21:39:55,NaN,3665.0,,,
13881,Panel Discussion,One Plus One,One Plus One,Series 2019 Murray Cook,Some of your favourite ABC presenters bring yo...,"Seven years ago, Murray Cook hung up his red s...","[ABC TV, ARTS & CULTURE, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC191...,NaN,"[abc1, news24, arts, panel, talks, people, int...",2019-03-15 13:30:00,NaN,1680.0,,,
13882,Panel Discussion,Q+A,Q+A,"The Vaccine, the Church, and the State of Poli...",Q+A is back to get more of the answers to your...,"The Government's ""women's ministry"" meets for ...","[ABC TV, ABC NEWS, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,"Host Virginia Trioli, David Speers, Stan Grant","[abc1, news, panel, current-affairs, politics,...",2021-04-08 21:31:57,NaN,3443.0,,,
13883,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 11 More Unseen Bits,Rob Brydon is back in the host's chair for ano...,More unseen clips from this series of Would I ...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-03-29 20:29:24,PG,1725.0,,,


In [258]:
tags2 = list(clean.tags2.dropna())

In [259]:
tags2

[['abc1',
  'abc2',
  'indigenous',
  'abcarts',
  'arts',
  'docs',
  'factual',
  'first-nations',
  'diversity',
  'bangarra',
  'dance',
  'performance',
  'biography',
  'inspirational',
  'revealing',
  'behind-the-scenes',
  'ns:be-inspired',
  'ns:be-uplifted',
  'a:older-sink-dink',
  'a:established-parents',
  'australia'],
 ['abc1',
  'docs',
  'factual',
  'arts',
  'indigenous',
  'music',
  'culture',
  'profiles',
  'short',
  'history',
  'indigenous-history',
  'compelling',
  'inspirational',
  'a:older-sink-dink',
  'ns:be-inspired',
  'ns:understand',
  'australia'],
 ['abc1',
  'abc2',
  'arts',
  'abcarts',
  'docs',
  'movies',
  'retro',
  'docuseries',
  'pop-culture',
  'history',
  'historical-events',
  'ns:be-entertained',
  'ns:be-informed',
  'ns:understand',
  'a:young-adults',
  'a:young-sink-dink',
  'a:older-sink-dink',
  'a:establish'],
 ['abc2',
  'docs',
  'factual',
  'arts',
  'ballet',
  'dance',
  'culture',
  'performance',
  'inspirational',


In [263]:
clean[clean['title'] == 'Would I Lie To You?'].sort_values(by = ['season', 'episode'])

,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
13877,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 1,Rob Brydon is back in the host's chair for ano...,"Rob Brydon, Lee Mack and David Mitchell are jo...","[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-01-25 20:29:51,PG,1741.0,13,1,
13788,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 10 The Unseen Bits,Rob Brydon is back in the host's chair for ano...,Previously unseen clips from this series of Wo...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-03-22 20:29:20,PG,1722.0,13,10,The Unseen Bits
13446,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 11 More Unseen Bits,Rob Brydon is back in the host's chair for ano...,More unseen clips from this series of Would I ...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-03-29 20:29:24,PG,1725.0,13,11,More Unseen Bits
13883,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 11 More Unseen Bits,Rob Brydon is back in the host's chair for ano...,More unseen clips from this series of Would I ...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-03-29 20:29:24,PG,1725.0,13,11,More Unseen Bits
13737,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 2,Rob Brydon is back in the host's chair for ano...,The guests joining Lee Mack and David Mitchell...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-01-25 21:00:47,PG,1737.0,13,2,
13705,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 3,Rob Brydon is back in the host's chair for ano...,The guests joining Lee Mack and David Mitchell...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-02-01 20:31:00,PG,1735.0,13,3,
13409,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 4,Rob Brydon is back in the host's chair for ano...,Host Rob Brydon and team captains Lee Mack and...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-02-08 20:30:35,PG,1737.0,13,4,
13489,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 5,Rob Brydon is back in the host's chair for ano...,This week the guests joining Lee Mack and Davi...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-02-15 20:30:08,PG,1736.0,13,5,
13600,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 6,Rob Brydon is back in the host's chair for ano...,This week the guests joining Lee Mack and Davi...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, comedy, panel, entertainment, cult-...",2022-02-22 20:00:38,PG,1727.0,13,6,
13570,Panel Discussion,Would I Lie To You?,Would I Lie To You?,Series 13 Episode 7,Rob Brydon is back in the host's chair for ano...,This week the guests joining Lee Mack and Davi...,"[ABC TV Plus, COMEDY, PANEL & DISCUSSION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW213...,Host Rob Brydon,"[abc2, uk, 

In [275]:
clean.loc[13446] == clean.loc[13883]

category             True
title                True
series               True
episode_name         True
description          True
description2         True
tags                 True
image               False
directors_actors     True
tags2                True
publication_date     True
rating               True
duration_sec         True
season               True
episode              True
episode_title        True
dtype: bool

In [272]:
clean.loc[13446, 'image']

'https://cdn.iview.abc.net.au/thumbs/i/zw/ZW2134A_5e71770aaa623.jpg'

In [274]:
clean.loc[13883, 'image']

'https://cdn.iview.abc.net.au/thumbs/i/zw/ZW2134A011S00_5efe8ca534e55_1280.jpg'

In [277]:
def allUnique(x):
    seen = set()
    return not any(i in seen or seen.add(i) for i in x)

In [278]:
allUnique(list(clean['title']))

False

In [282]:
# Finding TV shows

def unique_titles(title_list):
    not_unique = []
    s = set()
    for x in title_list:
        if x in s: not_unique.append(x)
        s.add(x)
    return set(not_unique)

unique_titles(list(clean['title']))

{'100 Bloody Acres',
 '13 Assassins',
 '199 Little Heroes',
 '30 Days Of Night',
 '7.30',
 '7.30 Mark Humphries Satire',
 '7.30: The Interviews',
 '72 Cutest Animals',
 "A Dog's World With Tony Armstrong",
 'A Raisin In The Sun',
 'A Royal Night Out',
 'A Secret',
 "A Stargazer's Guide To The Cosmos (2018)",
 'ABC Education Good Mornings',
 'ABC Late News',
 'ABC Late News Weekend',
 'ABC News ACT',
 'ABC News At Noon',
 'ABC News At Noon Weekend',
 'ABC News Mornings',
 'ABC News NSW',
 'ABC News NT',
 'ABC News QLD',
 'ABC News Regional',
 'ABC News SA',
 'ABC News Stories',
 'ABC News TAS',
 'ABC News VIC',
 'ABC News Video Lab',
 'ABC News WA',
 'ABC News With Auslan',
 'ABC SingSong',
 'Accidents Happen',
 'Accused',
 'Ackley Bridge',
 'Adam Hills: The Last Leg',
 'Adaptation',
 'Adoration',
 'Advice To My 12 Year Old Self',
 'Afternoon Briefing',
 'Against All Odds: Inside Statistics',
 'Ageless Friendships',
 'Alan Kohler Explains',
 'All-Round Champion',
 "Alva's World",
 'Amel

In [285]:
frame[frame['title'] == "Bangarra's World"]

,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_description
0,Arts,Bangarra's World,Bangarra's World,Return to Country (Yirrkala),Join the dancers and creatives during Bangarra...,"In Yirrkala, Northeast Arnhem Land, Bangarra s...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,G,626.0,,,
331,Arts,Bangarra's World,Bangarra's World,Taking on Stamping Ground,Join the dancers and creatives during Bangarra...,"For the first time ever, Bangarra performs a w...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,PG,611.0,,,
372,Arts,Bangarra's World,Bangarra's World,Being Bennelong,Join the dancers and creatives during Bangarra...,"Meet Beau Dean Riley Smith, the Wiradjuri man ...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,PG,549.0,,,
584,Arts,Bangarra's World,Bangarra's World,Touring Canada,Join the dancers and creatives during Bangarra...,"Bangarra tour Canada, a country with a colonia...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,G,501.0,,,
735,Arts,Bangarra's World,Bangarra's World,Unaipon,Join the dancers and creatives during Bangarra...,One of three works performed for Bangarra's 30...,"[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,PG,577.0,,,
841,Arts,Bangarra's World,Bangarra's World,Return to Country (Yirrkala),Join the dancers and creatives during Bangarra...,"In Yirrkala, Northeast Arnhem Land, Bangarra s...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,G,626.0,,,
868,Arts,Bangarra's World,Bangarra's World,Art of Bangarra,Join the dancers and creatives during Bangarra...,Bangarra's works are renowned for their theatr...,"[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,G,517.0,,,
913,Documentary,Bangarra's World,Bangarra's World,Return to Country (Yirrkala),Join the dancers and creatives during Bangarra...,"In Yirrkala, Northeast Arnhem Land, Bangarra s...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,G,626.0,,,
1322,Documentary,Bangarra's World,Bangarra's World,Taking on Stamping Ground,Join the dancers and creatives during Bangarra...,"For the first time ever, Bangarra performs a w...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous, abcarts, arts, docs, ...",2021-07-07 22:00:00,PG,611.0,,,
1397,Documentary,Bangarra's World,Bangarra's World,Being Bennelong,Join the dancers and creatives during Bangarra...,"Meet Beau Dean Riley Smith, the Wiradjuri man ...","[ABC TV, ABC TV Plus, INDIGENOUS, ARTS & CULTU...",https://cdn.iview.abc.net.au/thumbs/i/ar/AR194...,"Director Nel Minchin, Wayne Blair","[abc1, abc2, indigenous,